# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [5]:
import pandas as pd

In [6]:
df = pd.read_csv('data/bank-additional/bank-additional-full.csv', sep = ';')

In [7]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp.var.rate    41188 non-null  float64
 16  cons.price.idx  41188 non-null  float64
 17  cons.conf.idx   41188 non-null 

### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

#### Check Data for Null Values

In [13]:
print(df.shape)
# Check and list columns with NaNs
columns_with_nan = df.columns[df.isna().any()].tolist()
print(f"Columns with at least one NaN value: {columns_with_nan}")

(41188, 21)
Columns with at least one NaN value: []


In [14]:
## There are no columns with any Null values
## Check for "unknown" in the columns with categorical data along with their counts for each column
coulmns_unknown = {}
total_count = 0
for column in df.columns:
    if df[column].dtype == 'object':
        count = df[column].str.contains('unknown').sum()
        coulmns_unknown[column] = count
        total_count += count
for column, count in coulmns_unknown.items():
    if (count!=0 and column!='y'):
        print(f"Column '{column}' has {count} occurrences of 'unknown'")
print(total_count)

Column 'job' has 330 occurrences of 'unknown'
Column 'marital' has 80 occurrences of 'unknown'
Column 'education' has 1731 occurrences of 'unknown'
Column 'default' has 8597 occurrences of 'unknown'
Column 'housing' has 990 occurrences of 'unknown'
Column 'loan' has 990 occurrences of 'unknown'
12718


In [15]:
## Import libraries
from sklearn.compose import make_column_selector, make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.metrics import confusion_matrix, roc_curve, auc

import numpy as np

#### The total count of 12718 corresponds to the maximum number of rows that can have 'unknown' value 
#### (assuming that each row has eaxctly one column with 'unknown' value)
#### This corresponds to 12718 rows (at most) being deleted out of the total of 41188 rows or about 31% of the rows resulting in at least 28470 rows
#### The subsequent analysis will be performed after removing such values from dfbb

In [17]:
#select columns with object data type
str_columns = df.select_dtypes(include=['object']).columns
# Drop rows where any object type column contains 'unknown
df_cl = df[~df[str_columns].apply(lambda x: x.str.contains('unknown')).any(axis=1)]
print(df_cl.shape)

(30488, 21)


#### The actual reduction in number of rows is less (10700) than estimated earlier (12718)
#### the analysis now continues with df_cl 

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_cl.drop(['y'], axis = 1), df_cl['y'], random_state = 42, stratify = df_cl['y'])
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(22866, 20)
(22866,)
(7622, 20)
(7622,)


### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [22]:
## Baseline performance
baseline = df_cl['y'].value_counts(normalize = True).iloc[0]

print(f"baseline accuracy is {baseline}")

baseline accuracy is 0.8734256100760955


### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

#### The basic Logistic Regression model contains only the numeric features

In [25]:
int_columns = df.select_dtypes(include=['int64', 'float64']).columns
print(int_columns)

Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object')


In [26]:
from sklearn.compose import ColumnTransformer

preproc = ColumnTransformer(transformers=[('ssclr', StandardScaler(), int_columns)])

lgr_pipe = Pipeline([('preproc', preproc), ('lgr', LogisticRegression(random_state=42))])

%time lgr_pipe.fit(X_train, y_train)


CPU times: total: 250 ms
Wall time: 98.7 ms


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('ssclr', StandardScaler(),
                                                  Index(['age', 'duration', 'campaign', 'pdays', 'previous', 'emp.var.rate',
       'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'],
      dtype='object'))])),
                ('lgr', LogisticRegression(random_state=42))])

#### Improve Logistic Regression by adding the "object" dtypes in Problem 9
#### Create additional models using k-nearest neighbors, decision trees, and support vector machines, in Problem 9

### Problem 9: Score the Model

What is the accuracy of your model?

#### Scoring of Simple Logistic Regression Model

In [30]:
lgr_acc_train = lgr_pipe.score(X_train, y_train)
lgr_acc_test = lgr_pipe.score(X_test, y_test)

print(f"Simple Logistic Regression model training accuracy is {lgr_acc_train}")
print(f"Simple Logistic Regression model testing accuracy is {lgr_acc_test}")

Simple Logistic Regression model training accuracy is 0.9001137059389487
Simple Logistic Regression model testing accuracy is 0.8953030700603516


#### Simple Logistic Regression model testing accuracy 0.8953030700603516 is greater than the baseline accuracy of 0.8734256100760955

#### Logistic Regression with ALL features including the Categorical features

In [33]:
selector = make_column_selector(dtype_include=object) 

transformer = make_column_transformer((OneHotEncoder(drop = 'first'), selector),
                                     remainder = StandardScaler())
extractor = SelectFromModel(LogisticRegression(penalty='l1', solver = 'liblinear' ,random_state = 42))

lgr_pipe_all = Pipeline([('transformer', transformer), ('modelselector', extractor), ('lgr_all', LogisticRegression(random_state=42))])
print(lgr_pipe_all)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('modelselector',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=42,
                                                              solver='liblinear'))),
                ('lgr_all', LogisticRegression(random_state=42))])


In [34]:
%time lgr_pipe_all.fit(X_train, y_train)

CPU times: total: 2.53 s
Wall time: 1.83 s


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('modelselector',
                 SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                              random_state=42,
                                                              solver='liblinear'))),
                ('lgr_all', LogisticRegression(random_state=42))])

In [35]:
lgr_acc_all_train = lgr_pipe_all.score(X_train, y_train)
lgr_acc_all_test = lgr_pipe_all.score(X_test, y_test)
print(f"Logistic Regression model with prioritized fetaure set - training accuracy is {lgr_acc_all_train}")
print(f"Logistic Regression model with prioritized fetaure set - testing accuracy is {lgr_acc_all_test}")

Logistic Regression model with prioritized fetaure set - training accuracy is 0.9026502230385726
Logistic Regression model with prioritized fetaure set - testing accuracy is 0.8981894515875098


#### List feature names selected for Logistic Regression

In [37]:
feature_names = lgr_pipe_all.named_steps['transformer'].get_feature_names_out() 
selected_features =feature_names[ [int(i[1:]) for i in lgr_pipe_all.named_steps['modelselector'].get_feature_names_out()]]
clean_names = [i.split('__')[-1] for i in selected_features]
coef_df = pd.DataFrame({'feature': clean_names, 'coefs': lgr_pipe_all.named_steps['lgr_all'].coef_[0]})
coef_df['coefs'] = coef_df['coefs'].apply(abs)
coef_df = coef_df.sort_values(by = 'coefs', ascending = False)

print(feature_names.shape)
print(coef_df.shape)
print(coef_df)

(47,)
(42, 2)
                          feature     coefs
37                   emp.var.rate  2.636151
21                      month_mar  1.934591
38                 cons.price.idx  1.237569
33                       duration  1.211587
31               poutcome_success  0.730885
18                      month_aug  0.707402
17              contact_telephone  0.680490
20                      month_jun  0.549245
22                      month_may  0.531707
23                      month_nov  0.451913
30           poutcome_nonexistent  0.442079
40                      euribor3m  0.435560
41                    nr.employed  0.361667
3                     job_retired  0.336109
6                     job_student  0.274185
25                      month_sep  0.265989
35                          pdays  0.242870
0                 job_blue-collar  0.210873
24                      month_oct  0.203554
29                day_of_week_wed  0.195195
1                job_entrepreneur  0.149407
19                

#### 42 features are selected from the 47 features coming out of the first step of the pipeline
#### The 42 features are listed above in descending order of the coeficient of the fetaure
#### penalty = l1 was used as it gave higher score than l2 

#### K-Nearest Neighbors model with ALL features input to the pipeline and then selected using SelectFromModel class

In [182]:
from sklearn.neighbors import KNeighborsClassifier

knn_pipe_all = Pipeline([('transformer', transformer), ('knn_all', KNeighborsClassifier())])
print(knn_pipe_all)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('knn_all', KNeighborsClassifier())])


In [184]:
%time knn_pipe_all.fit(X_train, y_train)

CPU times: total: 109 ms
Wall time: 107 ms


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('knn_all', KNeighborsClassifier())])

In [186]:
knn_acc_all_train = knn_pipe_all.score(X_train, y_train)
knn_acc_all_test = knn_pipe_all.score(X_test, y_test)
print(f"KNN model with prioritized feature set - training accuracy is {knn_acc_all_train}")
print(f"KNN model with prioritized feature set - testing accuracy is {knn_acc_all_test}")

KNN model with prioritized feature set - training accuracy is 0.9200559783084055
KNN model with prioritized feature set - testing accuracy is 0.888743112044083


#### Simple Decision Tree Classifier

In [189]:
from sklearn.tree import DecisionTreeClassifier
dtc_pipe_all = Pipeline([('transformer', transformer), ('knn_all', DecisionTreeClassifier(random_state=42))])
print(dtc_pipe_all)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('knn_all', DecisionTreeClassifier(random_state=42))])


In [191]:
%time dtc_pipe_all.fit(X_train, y_train)

CPU times: total: 328 ms
Wall time: 329 ms


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('knn_all', DecisionTreeClassifier(random_state=42))])

In [193]:
dtc_acc_all_train = dtc_pipe_all.score(X_train, y_train)
dtc_acc_all_test = dtc_pipe_all.score(X_test, y_test)
print(f"Decision Tree model with prioritized feature set - training accuracy is {dtc_acc_all_train}")
print(f"Decision Tree model with prioritized feature set - testing accuracy is {dtc_acc_all_test}")

Decision Tree model with prioritized feature set - training accuracy is 1.0
Decision Tree model with prioritized feature set - testing accuracy is 0.8722120178430858


#### While the training accuracy is 1.0, the testing accuracy is 0.8758855943321963
#### which is only slightly higher than the baseline accuracy 0.8734256100760955
#### This is likely due to overfitting, which will be addressed later in this Notebook

#### Support Vector Machines Simple Model with default rbf

In [196]:
from sklearn.svm import SVC

svm_pipe_all = Pipeline([('transformer', transformer), ('svm_all', SVC())])
print(svm_pipe_all)

Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('svm_all', SVC())])


In [198]:
%time svm_pipe_all.fit(X_train, y_train)

CPU times: total: 12.5 s
Wall time: 12.8 s


Pipeline(steps=[('transformer',
                 ColumnTransformer(remainder=StandardScaler(),
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                ('svm_all', SVC())])

In [51]:
svm_acc_all_train = svm_pipe_all.score(X_train, y_train)
svm_acc_all_test = svm_pipe_all.score(X_test, y_test)
print(f"SVM model with prioritized feature set - training accuracy is {svm_acc_all_train}")
print(f"SVM model with prioritized feature set - testing accuracy is {svm_acc_all_test}")

SVM model with prioritized feature set - training accuracy is 0.9154202746435756
SVM model with prioritized feature set - testing accuracy is 0.8989766465494621


### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
| Simple Logistic Regression – Numeric features only, with SelectFromModel    | 0.0987 s   |0.9001137059389487     |0.8953030700603516     |
| Simple Logistic Regression – All features, with SelectFromModel             | 1.89 s     |0.9026502230385726     |0.8981894515875098     |
| Simple K-Nearest Neighbors – All features with SelectFromModel              | 0.107 s    |0.9200559783084055     |0.888743112044083      |
| Simple Decision Tree – All features with SelectFromModel                    | 0.329 s    |1.0                    |0.8722120178430858     |
| Simple SVM– All features with SelectFromModel and default kernel = ‘rbf’    | 12.8 s     |0.9154202746435756     |0.8989766465494621     |

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.

- More feature engineering and exploration.  For example, should we keep the gender feature?  Why or why not?
- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

### Logistic Regression with GridSearchCV

In [98]:
X_cl = df_cl.drop(['y'], axis = 1)
y_cl = df_cl['y']
print(X_cl.shape)
print(y_cl.shape)

(30488, 20)
(30488,)


In [154]:
lgr_pipe2 = Pipeline([('transformer', transformer), ('modelselector', extractor),
('lgr2', LogisticRegression())])
params = {'lgr2__C': [0.5, 1.0, 1.5, 2.0, 5.0], 'lgr2__max_iter': [1000], 'lgr2__fit_intercept': [True, False]}
lgr2_grid = GridSearchCV(estimator=lgr_pipe2, param_grid=params)
print(lgr2_grid)


GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('modelselector',
                                        SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                                                     random_state=42,
                                                                                     solver='liblinear'))),
                                       ('lgr2', LogisticRegression())]),
             param_grid={'lgr2__C': [0.5, 1.0, 1.5, 2.0, 5.0],

In [156]:
%time lgr2_grid.fit(X_cl, y_cl)

CPU times: total: 3min 9s
Wall time: 2min 11s


GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('modelselector',
                                        SelectFromModel(estimator=LogisticRegression(penalty='l1',
                                                                                     random_state=42,
                                                                                     solver='liblinear'))),
                                       ('lgr2', LogisticRegression())]),
             param_grid={'lgr2__C': [0.5, 1.0, 1.5, 2.0, 5.0],
                         'lgr2__fit_intercept': [True, False],
                         'lgr2__max_iter': [1000]})

In [172]:
lgr2_best_params = lgr2_grid.best_params_
print(lgr2_best_params)

{'lgr2__C': 0.5, 'lgr2__fit_intercept': True, 'lgr2__max_iter': 1000}


In [168]:
lgr2_preds_train = lgr2_grid.predict(X_train)
lgr2_preds_test = lgr2_grid.predict(X_test)

In [170]:
from sklearn.metrics import accuracy_score
lgr2_train_acc = accuracy_score(y_train, lgr2_preds_train)
lgr2_test_acc = accuracy_score(y_test, lgr2_preds_test)
print(lgr2_train_acc)
print(lgr2_test_acc)

0.9017318289162949
0.898583049068486


### KNN Classifier with GridSearchCV

In [205]:
knn_pipe2 = Pipeline([('transformer', transformer), ('knn2', KNeighborsClassifier())])
params = {'knn2__n_neighbors': [1, 2, 3, 4, 5]}
knn2_grid = GridSearchCV(estimator=knn_pipe2, param_grid=params)
print(knn2_grid)

GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('knn2', KNeighborsClassifier())]),
             param_grid={'knn2__n_neighbors': [1, 2, 3, 4, 5]})


In [209]:
%time knn2_grid.fit(X_cl, y_cl)

CPU times: total: 53.4 s
Wall time: 15 s


GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('knn2', KNeighborsClassifier())]),
             param_grid={'knn2__n_neighbors': [1, 2, 3, 4, 5]})

In [211]:
knn2_preds_train = knn2_grid.predict(X_train)
knn2_preds_test = knn2_grid.predict(X_test)

In [213]:
knn2_train_acc = accuracy_score(y_train, knn2_preds_train)
knn2_test_acc = accuracy_score(y_test, knn2_preds_test)
print(knn2_train_acc)
print(knn2_test_acc)

0.9190501180792443
0.9186565205982682


In [239]:
print(knn2_grid.best_params_)

{'knn2__n_neighbors': 4}


### Decision Tree Classfier with GridSearchCV

In [217]:
dtc_pipe2 = Pipeline([('transformer', transformer), ('dtc2', DecisionTreeClassifier())])
params = {'dtc2__max_depth': [5, 7, 10, 12, 15]}
dtc2_grid = GridSearchCV(estimator=dtc_pipe2, param_grid=params)
print(dtc2_grid)

GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('dtc2', DecisionTreeClassifier())]),
             param_grid={'dtc2__max_depth': [5, 7, 10, 12, 15]})


In [219]:
%time dtc2_grid.fit(X_cl, y_cl)

CPU times: total: 7.23 s
Wall time: 7.23 s


GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('dtc2', DecisionTreeClassifier())]),
             param_grid={'dtc2__max_depth': [5, 7, 10, 12, 15]})

In [221]:
dtc2_preds_train = dtc2_grid.predict(X_train)
dtc2_preds_test = dtc2_grid.predict(X_test)

In [223]:
dtc2_train_acc = accuracy_score(y_train, dtc2_preds_train)
dtc2_test_acc = accuracy_score(y_test, dtc2_preds_test)
print(dtc2_train_acc)
print(dtc2_test_acc)

0.9091664480013995
0.9060614012070323


In [241]:
print(dtc2_grid.best_params_)

{'dtc2__max_depth': 5}


### SVM with GridSearchCV

In [230]:
svm_pipe2 = Pipeline([('transformer', transformer), ('svm2', SVC())])
params = {
    'svm2__kernel': ['rbf'],
    'svm2__gamma': [0.1, 1.0, 10.0]
}
svm2_grid = GridSearchCV(estimator=svm_pipe2, param_grid=params)
print(svm2_grid)

GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('svm2', SVC())]),
             param_grid={'svm2__gamma': [0.1, 1.0, 10.0],
                         'svm2__kernel': ['rbf']})


In [232]:
%time svm2_grid.fit(X_cl, y_cl)

CPU times: total: 32min 5s
Wall time: 32min 29s


GridSearchCV(estimator=Pipeline(steps=[('transformer',
                                        ColumnTransformer(remainder=StandardScaler(),
                                                          transformers=[('onehotencoder',
                                                                         OneHotEncoder(drop='first'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001F8F46AD670>)])),
                                       ('svm2', SVC())]),
             param_grid={'svm2__gamma': [0.1, 1.0, 10.0],
                         'svm2__kernel': ['rbf']})

In [234]:
svm2_preds_train = svm2_grid.predict(X_train)
svm2_preds_test = svm2_grid.predict(X_test)

In [243]:
svm2_train_acc = accuracy_score(y_train, svm2_preds_train)
svm2_test_acc = accuracy_score(y_test, svm2_preds_test)
print(svm2_train_acc)
print(svm2_test_acc)

0.9975946820607015
0.9971136184728417


In [245]:
print(svm2_grid.best_params_)

{'svm2__gamma': 10.0, 'svm2__kernel': 'rbf'}


### Analysis of Improved Models using GridSearchCV

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
| Logistic Regression – All features with GridSearchCV         | 13.1 s     |0.9017318289162949     |0.898583049068486     |
| K-Nearest Neighbors – All features with GridSearchCV         | 3 s        |0.9190501180792443     |0.9186565205982682    |
| Decision Tree – All features with GridSearchCV               | 1.446 s    |0.9091664480013995     |0.9060614012070323    |
| SVM – All features with GridSearchCV                         | 641.67 s   |0.9975946820607015     |0.9971136184728417    |

##### Questions